In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Hyperparameters
LATENT_DIM = 100
IMG_SHAPE = (28, 28, 1)
BATCH_SIZE = 128
EPOCHS = 50
STEPS_PER_EPOCH = 500

# Build the Generator
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(256, input_dim=LATENT_DIM),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(momentum=0.8),
        layers.Dense(512),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(momentum=0.8),
        layers.Dense(1024),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(momentum=0.8),
        layers.Dense(np.prod(IMG_SHAPE), activation='tanh'),
        layers.Reshape(IMG_SHAPE)
    ])
    return model

# Build the Discriminator
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Flatten(input_shape=IMG_SHAPE),
        layers.Dense(512),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(256),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# Combined GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = tf.keras.Sequential([generator, discriminator])
    return model

# Loss functions
cross_entropy = tf.keras.losses.BinaryCrossentropy()

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Optimizers
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# Training step
@tf.function
def train_step(images, generator, discriminator, batch_size):
    noise = tf.random.normal([batch_size, LATENT_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

# Function to generate and save images
def generate_and_save_images(generator, epoch, test_noise):
    predictions = generator(test_noise, training=False)
    fig = plt.figure(figsize=(4, 4))

    for i in range(16):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    plt.savefig(f'image_at_epoch_{epoch:04d}.png')
    plt.close()

# Main training loop
def train_gan(dataset, generator, discriminator):
    test_noise = tf.random.normal([16, LATENT_DIM])

    for epoch in range(EPOCHS):
        print(f'Epoch {epoch + 1}/{EPOCHS}')
        for batch in dataset:
            gen_loss, disc_loss = train_step(batch, generator, discriminator, BATCH_SIZE)

        # Generate and save sample images every 10 epochs
        if (epoch + 1) % 10 == 0:
            generate_and_save_images(generator, epoch + 1, test_noise)

        print(f'Generator Loss: {gen_loss:.4f}, Discriminator Loss: {disc_loss:.4f}')

def main():
    # Load and preprocess MNIST dataset
    (x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
    x_train = x_train.reshape(-1, 28, 28, 1).astype('float32')
    x_train = (x_train - 127.5) / 127.5  # Normalize to [-1, 1]

    # Create dataset
    dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(60000).batch(BATCH_SIZE)

    # Initialize models
    generator = build_generator()
    discriminator = build_discriminator()

    # Train the GAN
    train_gan(dataset, generator, discriminator)

if __name__ == '__main__':
    main()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
Generator Loss: 4.0734, Discriminator Loss: 0.0671
Epoch 2/50
Generator Loss: 4.1152, Discriminator Loss: 0.0861
Epoch 3/50
Generator Loss: 3.4163, Discriminator Loss: 0.1908
Epoch 4/50
Generator Loss: 3.9177, Discriminator Loss: 0.3519
Epoch 5/50
Generator Loss: 2.9517, Discriminator Loss: 0.6817
Epoch 6/50
Generator Loss: 2.9772, Discriminator Loss: 0.6232
Epoch 7/50
Generator Loss: 2.4804, Discriminator Loss: 0.4628
Epoch 8/50
Generator Loss: 2.9621, Discriminator Loss: 0.7233
Epoch 9/50
Generator Loss: 1.9550, Discriminator Loss: 0.6662
Epoch 10/50
Generator Loss: 2.6956, Discriminator Loss: 0.8019
Epoch 11/50
Generator Loss: 2.2964, Discriminator Loss: 0.5583
Epoch 12/50
Generator Loss: 2.7128, Discriminator Loss: 0.9399
Epoch 13/50
Generator Loss: 1.8036, Discriminator Loss: 0.7408
Epoch 14/50
Generator Loss: 1.5189, Discriminator Loss: 0.8829
Epoch 15/50
Generator Loss: 1.8139, Discriminator Loss: 0.9026
Epoch 16/50
Generator Loss: 1.7583, Discriminator Loss: 0.8583
E